<a href="https://colab.research.google.com/github/devan1510/devan1510/blob/main/movie_review_analysis(using_a_Bidirectional_LSTM_model_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# sentiment analysis (us election tweets)

In [ ]:
# imports for the dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# make tensorflow imports
from tensorflow.keras.layers import SimpleRNN,LSTM,GRU,Bidirectional,Dense,Embedding

from tensorflow.keras.models import Sequential
# import the dataset
from tensorflow.keras.datasets import imdb

In [ ]:
# getting the most words in the review corpus
vocabulary_size= 500
(X_train,y_train),(X_test,y_test)= imdb.load_data(num_words= vocabulary_size)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# getting the words from the downloaded indexes
word_index= imdb.get_word_index()
# convert index as key and words as values using dictionary comprehension
word_index= {i: word for word,i in word_index.items()}

# print the review
print([word_index[i] for i in X_train[0]])


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
['the', 'as', 'you', 'with', 'out', 'and', 'and', 'and', 'and', 'their', 'becomes', 'and', 'had', 'and', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'and', 'never', 'more', 'and', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'and', 'and', 'movie', 'and', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'and', 'film', 'as', 'you', 'of', 'and', 'and', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'and', 'her', 'get', 'for', 'was', 'and', 'of', 'you', 'movie', 'and', 'movie', 'that', 'with', 'and', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', 'and', 'and', 'with', 'heart', 'had', 'and', 'they', 'of', 'here', 'that', 'with', 'her', 'and', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'and', 'they', 'is', 'you', 'that', "isn't", 'on

In [ ]:
print("Max length of a review:: ", len(max((X_train+X_test), key=len)))
print("Min length of a review:: ", len(min((X_train+X_test), key=len)))

Max length of a review::  2697
Min length of a review::  70


In [ ]:
from tensorflow.keras.preprocessing import sequence
# keeping a fixed length of all reviews to max 400 words
max_words= 400
X_train= sequence.pad_sequences(X_train,maxlen= max_words)
X_test= sequence.pad_sequences(X_test,maxlen= max_words)

# make validation set
X_valid, y_valid = X_train[:64], y_train[:64]
X_train, y_train = X_train[64:], y_train[64:]

In [ ]:
# build a simple rnn model
embedding_length= 32 # embedding with 32 feature vectors

# creating a rnn model
rnn_model= Sequential([
    Embedding(input_dim= vocabulary_size,
              output_dim= embedding_length,
              input_length= max_words),
    # to stack layers on top of each other use(return_sequences= True)
    SimpleRNN(units= 8,activation= 'tanh'),
    Dense(1,activation= 'sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
rnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# compiling the model
rnn_model.compile(loss= 'binary_crossentropy',
                  optimizer= 'adam',
                  metrics= ['accuracy'])
rnn_history= rnn_model.fit(X_train,y_train,
                           batch_size= 64,
                           epochs= 1,
                           validation_data= (X_valid,y_valid))

390/390 ━━━━━━━━━━━━━━━━━━━━ 42s 103ms/step - accuracy: 0.5793 - loss: 0.6705 - val_accuracy: 0.7344 - val_loss: 0.5650


In [ ]:
# evaluate the model
rnn_model.evaluate(X_test,y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.7247 - loss: 0.5734


[0.5733959078788757, 0.7238399982452393]

In [ ]:
# build a gru(gated recurrent units) model
gru_model= Sequential([
    Embedding(input_dim= vocabulary_size,
              output_dim= embedding_length,
              input_length= max_words),
    GRU(units= 8,activation= 'tanh'),
    Dense(units= 1,activation= 'sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
gru_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# compiling the model
gru_model.compile(loss= 'binary_crossentropy',
                  optimizer= 'adam',
                  metrics= ['accuracy'])
gru_history= gru_model.fit(X_train,y_train,
                           batch_size= 64,
                           epochs= 1,
                           validation_data= (X_valid,y_valid))

390/390 ━━━━━━━━━━━━━━━━━━━━ 84s 208ms/step - accuracy: 0.6174 - loss: 0.6428 - val_accuracy: 0.8125 - val_loss: 0.4645


In [ ]:
# evaluate the gru model
gru_model.evaluate(X_test,y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 37ms/step - accuracy: 0.7645 - loss: 0.4959


[0.4933997392654419, 0.7673599720001221]

In [ ]:
# build an lstm(long short term memory) model
lstm_model= Sequential([
    Embedding(input_dim= vocabulary_size,
              output_dim= embedding_length,
              input_length= max_words),
    LSTM(units= 8,activation= 'relu'),
    Dense(units= 1,activation= 'sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
lstm_model.compile(loss= 'binary_crossentropy',
                   optimizer= 'adam',
                   metrics= ['accuracy'])
# training the model
lstm_history= lstm_model.fit(X_train,y_train,
                             batch_size= 64,
                             epochs= 1,
                             validation_data= (X_valid,y_valid))

390/390 ━━━━━━━━━━━━━━━━━━━━ 64s 157ms/step - accuracy: 0.5255 - loss: nan - val_accuracy: 0.6094 - val_loss: nan


In [ ]:
# evaluate the lstm model
lstm_model.evaluate(X_test,y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 32ms/step - accuracy: 0.5073 - loss: nan


[nan, 0.5]

In [ ]:
# build a bi-directional lstm mdoel
bid_lstm_model= Sequential([
    Embedding(input_dim= vocabulary_size,
              output_dim= embedding_length,
              input_length= max_words),
    Bidirectional(LSTM(units= 8,
                       activation= 'tanh')),
    Dense(units= 1,activation= 'sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
bid_lstm_model.compile(loss= 'binary_crossentropy',
                       optimizer= 'adam',
                       metrics= ['accuracy'])
bid_lstm_history= bid_lstm_model.fit(X_train,y_train,
                   batch_size= 64,
                   epochs= 1,
                   validation_data= (X_valid,y_valid))

InvalidArgumentError: Graph execution error:

Detected at node sequential_4_1/bidirectional_1/forward_lstm_1_1/while/TensorListPushBack_13 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py",